<a href="https://colab.research.google.com/github/stefano2211/DataScienceRedSocial/blob/main/KingsLeagueJ6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [166]:
#Importamos las librerias
from textblob import TextBlob
import snscrape.modules.twitter as sntwitter
import plotly.express as px
import neattext as nt
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [167]:
#Cargamos las stopwords de nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [168]:
#Importamos los datos de twitter
query = "@KingsLeague"
tweets = []
limit = 3000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.content, tweet.hashtags, tweet.username, tweet.mentionedUsers])

<ipython-input-168-5f123059cd7e>:13: FutureWarning:

content is deprecated, use rawContent instead

<ipython-input-168-5f123059cd7e>:13: FutureWarning:

username is deprecated, use user.username instead



In [169]:
#Colocamos los datos en un dataframe
df = pd.DataFrame(tweets, columns=['Date', 'Tweet', 'Hashtag', 'Username', 'Mention'])
df.head(100)

,Date,Tweet,Hashtag,Username,Mention
0,2023-02-12 23:13:10+00:00,@6cichero6 @KingsLeague @PorcinosFC @IbaiLlano...,None,mickox,"[https://twitter.com/6cichero6, https://twitte..."
1,2023-02-12 23:12:37+00:00,@pedrosaezarenas @KingsLeague @futbolmahou Ana...,None,MigueCaballer11,"[https://twitter.com/pedrosaezarenas, https://..."
2,2023-02-12 23:12:26+00:00,@6cichero6 @KingsLeague @PorcinosFC @IbaiLlano...,None,DanielFloresJA,"[https://twitter.com/6cichero6, https://twitte..."
3,2023-02-12 23:11:34+00:00,Que bien me lo he pasado hoy en la @KingsLeagu...,None,pedrosaezarenas,"[https://twitter.com/KingsLeague, https://twit..."
4,2023-02-12 23:09:58+00:00,@JohanMM_ @loopez______ @KingsLeague @Porcinos...,None,Kanyentin,"[https://twitter.com/JohanMM_, https://twitter..."
...,...,...,...,...,...
95,2023-02-12 22:34:44+00:00,@KingsLeague @3gerardpique Era obvio que iba a...,None,Ramon_Cuerda,"[https://twitter.com/KingsLeague, https://twit..."
96,2023-02-12 22:34:32+00:00,@El_WillyMei @6cichero6 @KingsLeague @Porcinos...,None,iamdanielypunto,"[https://twitter.com/El_WillyMei, https://twit..."
97,2023-02-12 22:33:26+00:00,@6cichero6 @KingsLeague @PorcinosFC @IbaiLlano...,None,EmpanadadC,"[https://twitter.com/6cichero6, https://twitte..."
98,2023-02-12 22:33:21+00:00,Entre 400k y 700k esta la audiencia regular de...,None,Liderdragon1,[https://twitter.com/KingsLeague]


In [170]:
#Convertimos la columna Date a formato tiempo
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

In [171]:
#Filtamos los datos del periodo de tiempo en el cual se jugo la kings leage 
Df = df[df['Date'] > '2023-02-11']
DfToday = Df[Df['Date'] < '2023-02-13']
dt = DfToday

In [172]:
dt.head()

,Date,Tweet,Hashtag,Username,Mention
0,2023-02-12 23:13:10+00:00,@6cichero6 @KingsLeague @PorcinosFC @IbaiLlano...,None,mickox,"[https://twitter.com/6cichero6, https://twitte..."
1,2023-02-12 23:12:37+00:00,@pedrosaezarenas @KingsLeague @futbolmahou Ana...,None,MigueCaballer11,"[https://twitter.com/pedrosaezarenas, https://..."
2,2023-02-12 23:12:26+00:00,@6cichero6 @KingsLeague @PorcinosFC @IbaiLlano...,None,DanielFloresJA,"[https://twitter.com/6cichero6, https://twitte..."
3,2023-02-12 23:11:34+00:00,Que bien me lo he pasado hoy en la @KingsLeagu...,None,pedrosaezarenas,"[https://twitter.com/KingsLeague, https://twit..."
4,2023-02-12 23:09:58+00:00,@JohanMM_ @loopez______ @KingsLeague @Porcinos...,None,Kanyentin,"[https://twitter.com/JohanMM_, https://twitter..."


In [173]:
#Convertimos el None en Nan y luego remplazamos el Nan por un 1
dt['Hashtag'].replace('None', np.nan, inplace=True)
dt['Hashtag'] = dt['Hashtag'].fillna('1')



In [174]:
dt['Mention'].replace('None', np.nan, inplace=True)
dt['Mention'] = dt['Mention'].fillna('1')

In [175]:
dt.head()

,Date,Tweet,Hashtag,Username,Mention
0,2023-02-12 23:13:10+00:00,@6cichero6 @KingsLeague @PorcinosFC @IbaiLlano...,1,mickox,"[https://twitter.com/6cichero6, https://twitte..."
1,2023-02-12 23:12:37+00:00,@pedrosaezarenas @KingsLeague @futbolmahou Ana...,1,MigueCaballer11,"[https://twitter.com/pedrosaezarenas, https://..."
2,2023-02-12 23:12:26+00:00,@6cichero6 @KingsLeague @PorcinosFC @IbaiLlano...,1,DanielFloresJA,"[https://twitter.com/6cichero6, https://twitte..."
3,2023-02-12 23:11:34+00:00,Que bien me lo he pasado hoy en la @KingsLeagu...,1,pedrosaezarenas,"[https://twitter.com/KingsLeague, https://twit..."
4,2023-02-12 23:09:58+00:00,@JohanMM_ @loopez______ @KingsLeague @Porcinos...,1,Kanyentin,"[https://twitter.com/JohanMM_, https://twitter..."


In [176]:
#Extraemos los hashtags para convertirlos en strings
def textl(text):
  leters= " ".join([str(_) for _ in text])
  return leters


In [177]:
dt['TokenMention'] = dt['Mention'].apply(textl)

In [178]:
dt['Token'] = dt['Hashtag'].apply(textl)
dt.head()

,Date,Tweet,Hashtag,Username,Mention,TokenMention,Token
0,2023-02-12 23:13:10+00:00,@6cichero6 @KingsLeague @PorcinosFC @IbaiLlano...,1,mickox,"[https://twitter.com/6cichero6, https://twitte...",https://twitter.com/6cichero6 https://twitter....,1
1,2023-02-12 23:12:37+00:00,@pedrosaezarenas @KingsLeague @futbolmahou Ana...,1,MigueCaballer11,"[https://twitter.com/pedrosaezarenas, https://...",https://twitter.com/pedrosaezarenas https://tw...,1
2,2023-02-12 23:12:26+00:00,@6cichero6 @KingsLeague @PorcinosFC @IbaiLlano...,1,DanielFloresJA,"[https://twitter.com/6cichero6, https://twitte...",https://twitter.com/6cichero6 https://twitter....,1
3,2023-02-12 23:11:34+00:00,Que bien me lo he pasado hoy en la @KingsLeagu...,1,pedrosaezarenas,"[https://twitter.com/KingsLeague, https://twit...",https://twitter.com/KingsLeague https://twitte...,1
4,2023-02-12 23:09:58+00:00,@JohanMM_ @loopez______ @KingsLeague @Porcinos...,1,Kanyentin,"[https://twitter.com/JohanMM_, https://twitter...",https://twitter.com/JohanMM_ https://twitter.c...,1


In [212]:
tokenHashtag = dt['Token']

In [213]:
tokenMention = dt['TokenMention']

In [181]:
tokens = [token for line in tokenHashtag  for token in line.split()]
tokensMention = [token for line in tokenMention  for token in line.split()]

In [182]:
#Creamos una funcion que cuente los datos ya toquenizados 
def get_tokens(docx,num=40):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [183]:
tokensTotal = get_tokens(tokens)
TokenMentionTotal= get_tokens(tokensMention)

In [184]:
#Insertamos los datos en un dataframe
data = pd.DataFrame(tokensTotal.items(),columns=['Hashtags','scores'])
dataM = pd.DataFrame(TokenMentionTotal.items(),columns=['Mention','scores'])

In [185]:
dataM.head()

,Mention,scores
0,https://twitter.com/KingsLeague,2964
1,https://twitter.com/JijantesFC,654
2,https://twitter.com/PIO_FC,639
3,https://twitter.com/Aaronrv22_,440
4,https://twitter.com/DjMaRiiO,282


In [186]:
fig = px.bar(dataM, title="Menciones mas usados en la Kings League",x=dataM['Mention'], y=dataM['scores'])
fig.show()

#Conclusiones de los hashtags

Las menciones mas utilizados de Yummy Venezuela


*   KingsLeague
*   Jijantes
*   PioFc
*   DjMario
*   PorcinosFC

In [187]:
#Ploteamos los datos
fig = px.bar(data, title="Hashtags mas usados en la Kings League",x=data['Hashtags'], y=data['scores'])
fig.show()

#Conclusiones de los hashtags

Los hashtags mas utilizados de la Kings League


*   KingsLeagueJ6
*   KingsLeague
*   SimyoMVP
*   Puropinchepio
*   Vamos1K



In [188]:
#Removemos caracteres que puedan afectar la prediccion del modelo
dt['Tweet'] = dt['Tweet'].apply(nt.remove_hashtags)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_emojis)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_numbers)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_special_characters)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_urls)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_currency_symbols)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_userhandles)

In [189]:
#Creamos una funcion para borrar caracteres especiales
def clean_text(text):
  text = re.sub(r'@[A-Za-z09]+', '', text)
  text = re.sub(r'#', '', text)
  text = re.sub(r'RT[\s]+', '', text)
  text = re.sub(r'https?:\/\/?', '', text)


  return text

dt['Tweet'] = dt['Tweet'].apply(clean_text)

In [190]:
#Creamos dos fuciones una para clacular el sentimiento y el otro para calcular la subjetividad
def get_subjectivity(text):
  return TextBlob(text).translate(from_lang="es", to="en").sentiment.subjectivity

def get_polarity(text):
  return TextBlob(text).translate(from_lang="es", to="en").sentiment.polarity

In [191]:
dt['Subjectivity'] = dt['Tweet'].apply(get_subjectivity)

In [192]:
dt['Polarity'] = dt['Tweet'].apply(get_polarity)

In [ ]:
#Creamos una funcion que tome la polaridad y la procese como Positivo, Neutro o negativo
def analysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
     return 'Neutral'
  elif score > 0:
    return 'Positive'

dt['Analysis'] = dt['Polarity'].apply(analysis)

dt.head(800)

In [194]:
count = dt['Analysis'].value_counts()
count

Neutral     1941
Positive     704
Negative     355
Name: Analysis, dtype: int64

In [195]:
#Graficamos la polaridad
fig = px.scatter(dt, y=dt['Polarity'])
fig.show()

In [214]:
fig = px.bar(count, title="Sentimientos de los tweets de la kings league jornada 6", color=['red','blue','green'])
fig.show()

#Conclusiones de el analisis de emociones

Podemos concluir que hay mas comentarios negativos que positivos pero lo que mas se resalta es los comentarios neutros con una alta diferencia.



1.   Neutros: 1941
2.   Negativos: 355
3.   Positivos: 704


In [197]:
# Creamos tres sets de datos con los diferentes tipos de comentarios
positive_tweet = dt[dt['Analysis'] == 'Positive']
neutral_tweet = dt[dt['Analysis'] == 'Neutral']
negative_tweet = dt[dt['Analysis'] == 'Negative']

In [198]:
print(stopwords.words('spanish'))

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [199]:
#Eliminamos las preposiciones, adverbios y adjetivos
def textdrop(text):

  sw = stopwords.words('spanish')

  text = [word.lower() for word in text.split() if word.lower() not in sw]

  return " ".join(text)

In [ ]:
positive_tweet['Tweet'] = positive_tweet['Tweet'].apply(textdrop)
neutral_tweet['Tweet'] = neutral_tweet['Tweet'].apply(textdrop)
negative_tweet['Tweet'] = negative_tweet['Tweet'].apply(textdrop)

In [201]:
tokenPositive = positive_tweet['Tweet']
tokenNeutral = neutral_tweet['Tweet']
tokenNegative = negative_tweet['Tweet']

In [217]:
#Tokenizamos los tweets
pos_tokens = [token for line in tokenPositive  for token in line.split()]
neut_tokens = [token for line in tokenNeutral  for token in line.split()]
neg_tokens = [token for line in tokenNegative  for token in line.split()]

In [218]:
word_pos = get_tokens(pos_tokens)
word_neu = get_tokens(neut_tokens)
word_neg = get_tokens(neg_tokens)

In [ ]:
word_neu

In [220]:
#Creamos 3 sets de datos con las palabras y el contador 
pos_df = pd.DataFrame(word_pos.items(),columns=['words','scores'])
neut_df = pd.DataFrame(word_neu.items(),columns=['words','scores'])
neg_df = pd.DataFrame(word_neg.items(),columns=['words','scores'])

In [224]:
fig = px.bar(data, title="Palabras mas usadas en los tweets positivos de la kings league",x=pos_df['words'], y=pos_df['scores'])
fig.show()

In [223]:
fig = px.bar(data, title="Palabras mas usadas en los tweets negativos de la kings league",x=neg_df['words'], y=neg_df['scores'])
fig.show()

#Conclusiones de la tokenzacion

Podemos observar que en los tweets positivos las palabras claves que se repiten comunmente es:

*   KingsLeague
*   PioFc
*   JijanteFc
*   aaronrv

En los tweets negativo los mas resaltantes son:

*   KingsLeague
*   JijantesFc
*   PioFc
*   DjMario

In [209]:
fig = px.histogram(dt, x=dt["Date"], title="Tweets durante durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

#Tweets durante el Tiempo

Podemos ver que hay tres picos de comentarios que abarcan todo el dia de hoy



1.   A las 4 de la tarde comenzando los partidos
2.   A las 5:15 de la tarde en el partido entre ElBarrio y Troncos
3.   A las 7 de la tarde en el partido de Aniquiladores vs KunisSports



In [210]:
fig = px.histogram(dt, x=positive_tweet["Date"], title="Tweets positivos durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

#Tweets Positivos durante el tiempo

Podemos observar tres puntos principales 

1.   A las 5 cuando jugaba ElBarrio vs LosTroncos
2.   A las 3:30 previo a que comenzara los partidos 
3.   A las 4 cuando comenzo JijantesFc vs PioFc



In [211]:
fig = px.histogram(dt, x=negative_tweet["Date"], title="Tweets negativos durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

#Tweets Negativos durante el tiempo

Podemos observar dos puntos principales 

1.   A las 4:30 durante el Jijantes vs Pio
2.   A las 8 durante el PorcinosFC vs XBUYERTEAM 